In [1]:
using Flux
using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!
#using Flux: gpu
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using Flux: @epochs

using CSV, DataFrames
file = "BostonHousing.csv"

df = DataFrame(CSV.File(file)) #|> DataFrame!
#df[:medv]
#
lstat = df[:lstat]
medv = df[:medv]


try
    rawdata = readdlm("housing.data")
catch
    isfile("housing.data") ||
      download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data",
               "housing.data")
    rawdata = readdlm("housing.data")
end
rawdata = readdlm("housing.data")
summary(rawdata)

"505×14 Array{Float64,2}"

In [2]:
using Flux
using Flux: crossentropy
using Flux: @epochs
@doc(Conv)

```
Conv(size, in=>out)
Conv(size, in=>out, relu)
```

Standard convolutional layer. `size` should be a tuple like `(2, 2)`. `in` and `out` specify the number of input and output channels respectively.

Example: Applying Conv layer to a 1-channel input using a 2x2 window size,          giving us a 16-channel output. Output is activated with ReLU.

```
size = (2,2)
in = 1
out = 16 
Conv((2, 2), 1=>16, relu)
```

Data should be stored in WHCN order (width, height, # channels, # batches).  In other words, a 100×100 RGB image would be a `100×100×3×1` array,  and a batch of 50 would be a `100×100×3×50` array.

Takes the keyword arguments `pad`, `stride` and `dilation`.


In [45]:


# This replicates the housing data example from the Knet.jl readme. Although we
# could have reused more of Flux (see the mnist example), the library's
# abstractions are very lightweight and don't force you into any particular
# strategy.
# https://github.com/FluxML/model-zoo/blob/master/other/housing/housing.jl


print(summary(df))
# The last feature is our target -- the price of the house.
split_ratio = 0.5 # For the train test split

x = rawdata[12:12,:] #|> cpu
y = rawdata[14:14,:] #|> cpu
Xs = x
Ys = y
# Normalise the data
x = (x .- mean(x, dims = 2)) ./ std(x, dims = 2)

# Split into train and test sets
split_index = floor(Int,size(x,2)*split_ratio)
x_train = x[:,1:split_index]
y_train = y[:,1:split_index]
x_test = x[:,split_index+1:size(x,2)]
y_test = y[:,split_index+1:size(x,2)]

# The model
W = param(randn(1,1)/10) #|> gpu
b = param([0.]) #|> gpu

predict(x) = W*x .+ b
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)
loss(x, y) = meansquarederror(predict(x), y)

η = 0.1
θ = Params([W, b])

for i = 1:12
  g = gradient(() -> loss(x_train, y_train), θ)
  for x in θ
    update!(x, -g[x]*η)
  end
  @show loss(x_train, y_train)
end

# Predict the RMSE on the test set


# https://github.com/FluxML/model-zoo/blob/master/text/char-rnn/char-rnn.jl
m = Chain(
  LSTM(1, 505), #(inputs,outputs)
  LSTM(505, 100),
  Dense(505, 1), #(inputs,outputs)
  softmax)

  

#modelReg = Chain(Dense(1, 505))
#loss(x, y) = Flux.mse(modelReg(x), y)
#opt = SGD(Flux.params(modelReg), 0.1)
#@epochs 100 Flux.train!(loss, regData, opt)

#m = gpu(m)

#function loss(xs, ys)#
#  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
#  Flux.truncate!(m)
#  return l
#end
opt = SGD(params(m),0.1)
#
#opt = SGD()
#opt() # updates the weights

tx, ty = (Xs[5], Ys[5])
evalcb = () -> @show loss(tx, ty)
@epochs 40 Flux.train!(loss, zip(Xs[5], Ys[5]), opt)
opt = ADAM(0.1)
@epochs 40 Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb = throttle(evalcb, 30))


506×14 DataFrameloss(x_train, y_train) = 959.6566741388207 (tracked)
loss(x_train, y_train) = 914.0046169440519 (tracked)
loss(x_train, y_train) = 885.9537371725115 (tracked)
loss(x_train, y_train) = 868.2822400852872 (tracked)
loss(x_train, y_train) = 856.7331343340478 (tracked)
loss(x_train, y_train) = 848.7970686590787 (tracked)
loss(x_train, y_train) = 842.9948599242399 (tracked)
loss(x_train, y_train) = 838.4545703917364 (tracked)
loss(x_train, y_train) = 834.6622106899129 (tracked)
loss(x_train, y_train) = 831.3147885935798 (tracked)
loss(x_train, y_train) = 828.233686809572 (tracked)
loss(x_train, y_train) = 825.3136024407166 (tracked)


┌ Warning: SGD(params) is deprecated; use Descent(η::Float64) instead
│   caller = top-level scope at In[45]:56
└ @ Core In[45]:56
┌ Info: Epoch 1
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = top-level scope at train.jl:106
└ @ Core /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:106
┌ Info: Epoch 2
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 3
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 4
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 5
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 6
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 7
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Ep

loss(tx, ty) = 183.8715772123636 (tracked)


┌ Info: Epoch 2
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105


loss(tx, ty) = 183.8715772123636 (tracked)


...

loss(tx, ty) = 183.8715772123636 (tracked)


┌ Info: Epoch 38
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105


loss(tx, ty) = 183.8715772123636 (tracked)


┌ Info: Epoch 39
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105
┌ Info: Epoch 40
└ @ Main /home/russell/.julia/packages/Flux/qXNjB/src/optimise/train.jl:105


loss(tx, ty) = 183.8715772123636 (tracked)


In [1]:
#print(varinfo())
#using MLBase

err = meansquarederror(predict(x_test),y_test)
println(err)
collect(Kfold(10, 3))
using Flux: argmax

predicted = m(x_test[1])#, (28, 28, 1, 10000)))

#accuracy = mean(argmax(predicted)-1 .== y_test[1])
#println(accuracy)
print(predicted)
m(x_test[1])
y_test[1]
#print(x_test)

UndefVarError: UndefVarError: predict not defined

In [51]:
size(Xs)


(1, 14)

In [49]:
accuracy(x, y) = mean(onecold(m(x_test), 1:10) .== onecold(y_test, 1:10))
accuracy()

accuracy (generic function with 1 method)

In [5]:
summary(x)

"1×14 Array{Float64,2}"

In [6]:
summary(lstat)

"506-element Array{Union{Missing, Float64},1}"

In [7]:
#print(loss)
#params(m), zip(Xs, Ys), 
#print(opt)

In [8]:
using StatsPlots



In [9]:
p1 = scatter(loss,1:size(loss))
plot(p1)

MethodError: MethodError: no method matching size(::typeof(loss))
Closest candidates are:
  size(!Matched::BitArray{1}) at bitarray.jl:77
  size(!Matched::BitArray{1}, !Matched::Any) at bitarray.jl:81
  size(!Matched::Core.Compiler.StmtRange) at show.jl:1561
  ...

In [10]:
loss.size

ErrorException: type #loss has no field size

In [11]:
shape(loss)

UndefVarError: UndefVarError: shape not defined

In [47]:
#varinfo()

In [13]:

using Flux: argmax

predicted = model(reshape(Float64.(test_x), (28, 28, 1, 10000)))

accuracy = mean(argmax(predicted)-1 .== test_y)
println(accuracy)

UndefVarError: UndefVarError: test_x not defined